In [1]:
# Dependencies
import matplotlib.pyplot as plt
from citipy import citipy as cp
import pandas as pd
import requests as req
import json

In [2]:
#from citipy, retrieve list of cities based on lat and lng 
list_of_cities = []
counter = 0
Duplicates  = 'no'

for x in range(-90,90,1):
    for y in range(-180,180,1):
        city = cp.nearest_city(x, y)
        citdict = {}
        citdict['city'] = city.city_name
        citdict['country'] = city.country_code
        citdict['lat'] = x
        citdict['long'] = y
        if len(list_of_cities) == 0:
            list_of_cities.append(citdict)
            counter+=1
            continue
        else:
            #Get rid of duplicates
            for city in list_of_cities:
                if city['city'] == citdict['city']:
                    Duplicates = 'yes'
        if Duplicates == 'no':
            list_of_cities.append(citdict)
            counter+=1
        else:
            Duplicates = 'no'

print(len(list_of_cities))

7957


In [4]:
print(list_of_cities[0])

{'city': 'vaini', 'country': 'to', 'lat': -90, 'long': -180}
